In [7]:
import json
import os
import nltk
from nltk.tokenize import word_tokenize
import nltk.data
import progressbar
import pandas as pd


## Loading all the json files
IT_train_path = 'IT_train.json'
IT_test_path = 'IT_test.json'
CL_train_path = 'CL_train.json'
CL_test_path = 'CL_test.json'
IT_decision = 'IT_decision.json'
CL_decision = 'CL_decision.json'
IT_threshold = 'it_case_decision.json'
CL_threshold = 'cl_case_decision.json'

train_it = {}
test_it = {}
train_cl = {}
test_cl = {}

with open(IT_train_path, 'r') as f:
    train_it = json.load(f)
    f.close()
    
with open(IT_test_path, 'r') as f:
    test_it = json.load(f)
    f.close()

with open(CL_train_path, 'r') as f:
    train_cl = json.load(f)
    f.close()

with open(CL_test_path, 'r') as f:
    test_cl = json.load(f)
    f.close()
    
with open(IT_decision, 'r') as f:
    decision_it = json.load(f)
    f.close()

with open(CL_decision, 'r') as f:
    decision_cl = json.load(f)
    f.close()
    
with open(IT_threshold, 'r') as f:
    threshold_it = json.load(f)
    f.close()

with open(CL_threshold, 'r') as f:
    threshold_cl = json.load(f)
    f.close()
    

In [8]:
all_it_cl = {**test_it, **test_cl}
all_decision = {**decision_cl, **decision_it}
all_threshold = {**threshold_it, **threshold_cl}

In [9]:
## Here we get the entire case text and the ROD(last 512 tokens) RPC text for each case

text = []
text_full = []
labels = []
name = []

for case in all_it_cl.keys():
    if(all_threshold[case] != 1):
        sentences = all_it_cl[case]['sentences'][:all_decision[case]['decision_sentence']+1]
    else:
        sentences = all_it_cl[case]['sentences'][:all_decision[case]['decision_sentence']]
    label = all_decision[case]['decision']
    start_idx = 0
    count = 512
    for idx, sent in enumerate(reversed(sentences)):
        if(len(word_tokenize(sent)) < count):
            count -= len(word_tokenize(sent))
            start_idx += 1
        else:
            break
    consider = len(sentences) - start_idx
    rpc = 'RulingByPresentCourt'
    rod = 'RatioOfTheDecision'
    all_text = ""
    for idx, sent in enumerate(sentences):
        if((all_it_cl[case]['complete'][idx] == rpc)):
            all_text += sent + " "
        if(all_it_cl[case]['complete'][idx] == rod and idx >= consider):
            all_text += sent + " "
    tokens = word_tokenize(all_text)
    num_tokens = len(tokens)
    if(num_tokens < 20):
        continue
    text.append(all_text)
    final_text = (" ").join(sentences)
    text_full.append(final_text)
    labels.append(label)
    name.append(case)

In [10]:
## Converting both data in dataframe

df = {}
df['text'] = text
df['label'] = labels
df['name'] = name

df_full = {}
df_full['text'] = text_full
df_full['label'] = labels
df_full['name'] = name

df = pd.DataFrame.from_dict(df)
df_full = pd.DataFrame.from_dict(df_full)

df.to_csv('IT/it_cl_ljp_rod_rpc_final.csv')
df_full.to_csv('IT/it_cl_ljp_final.csv')